In [33]:
import pandas as pd
import os
import re

In [179]:
dirpath = 'section2'
lst = []

for _, _, files in os.walk(dirpath):
    for fn in files:
        if os.path.splitext(fn)[1] == '.tex':
            filepath = os.path.join(dirpath, fn)
            print(filepath)
            with open(filepath, 'r') as f:
                for l in f.readlines():
                    if l[0] != '%':
                        lst.append(l.strip())

section2\0_abstract.tex
section2\1_intro.tex
section2\2_NI.tex
section2\3_AI.tex
section2\4_disc.tex
section2\diversity.tex


In [228]:
tmp[0]

('e', 'kandel2000principles, bermudez2000cognitive')

In [232]:
output = []

for line in lst:
    tmp = re.findall('cit(e|et)\{(.*?)\}', line)
    if tmp:
        for t in tmp:
            output += t[1].split(',')

final = []
for i in output:
    final += [i.strip()]

print(f'Length of the raw list: {len(final)}')

df = pd.DataFrame(final, columns=['text']).drop_duplicates('text')

print(f'Length after removing duplicates: {df.shape[0]}')

Length of the raw list: 104
Length after removing duplicates: 92


In [230]:
df.to_excel('output.xlsx', index=False)